# CPC Wrfcf Root Mean Sq Error 2017-02-05 to 2017-02-17  

In [1]:
# imports
import numpy
import Nio
import Ngl
from IPython import display
import netCDF4 as nc
from datetime import date
from datetime import timedelta
import xarray as xr
import xesmf as xe
import pandas as pd

In [2]:
# read in daily difference datasets
dif_05 = xr.open_dataarray('./dif_05_cpc-Copy1.nc')
dif_06 = xr.open_dataarray('./dif_06_cpc-Copy1.nc')
dif_07 = xr.open_dataarray('./dif_07_cpc-Copy1.nc')
dif_08 = xr.open_dataarray('./dif_08_cpc-Copy1.nc')
dif_09 = xr.open_dataarray('./dif_09_cpc-Copy1.nc')
dif_10 = xr.open_dataarray('./dif_10_cpc-Copy1.nc')
dif_11 = xr.open_dataarray('./dif_11_cpc-Copy1.nc')
dif_12 = xr.open_dataarray('./dif_12_cpc-Copy1.nc')
dif_13 = xr.open_dataarray('./dif_13_cpc-Copy1.nc')
dif_14 = xr.open_dataarray('./dif_14_cpc-Copy1.nc')
dif_15 = xr.open_dataarray('./dif_15_cpc-Copy1.nc')

dif_arr = [dif_05, dif_06, dif_07, dif_08, dif_09, dif_10, dif_11, dif_12, dif_13, dif_14, dif_15]

In [3]:
# rmse
def rmse():
    sq_arr = []
    for i in dif_arr:
        sq_dif = numpy.square(i)
        sq_arr.append(sq_dif)
    sum_arr = sq_arr[0]
    for j in sq_arr[1:]:
        sum_arr += j
    mean_sq = sum_arr / 11
    rmse = numpy.sqrt(mean_sq)
    return rmse

rm = rmse()
rm

<xarray.DataArray (lat: 360, lon: 720)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lat      (lat) float32 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon      (lon) float32 -359.8 -359.2 -358.8 -358.2 ... -1.25 -0.75 -0.25

In [4]:
# for reference
numpy.unique(rm), max(numpy.unique(rm)), min(numpy.unique(rm))

(array([0.46464337, 0.78253687, 0.79079631, ...,        nan,        nan,
               nan]),
 10.068399522798522,
 0.4646433705001326)

In [5]:
# convert array to masked array to overlook NaNs
import numpy.ma as ma
m_out = ma.masked_invalid(rm)
m_out

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20)

In [6]:
# numerical data, average rmse over entire region
def overall_rmse():
    sq_arr = []
    for i in dif_arr:
        sq_dif = numpy.square(i.data)
        sq_arr.append(sq_dif)
    arr_sq = [x for x in sq_arr[0].flatten() if str(x) != 'nan']
    div = sum(arr_sq) / len(arr_sq)
    return numpy.sqrt(div) / 11
overall_rmse()

0.327751049932949

# workstation
wks_type2 = "png"
wks2 = Ngl.open_wks(wks_type2,"17_rmse_NG-GR-NQ-BD")

# borrow RHS plot end coords
fn = '/cw3e/mead/projects/cwp103/production/output/2017020500/cf/NG-GR-NQ-BD/wrfcf_d02_2017-02-05_22_00_00.nc'
da_input = xr.open_dataset(fn)

# defining variables
temp = m_out
lat = rm['lat'].data.flatten()
lon = rm['lon'].data.flatten()
lat2 = da_input['lat'].data.flatten()
lon2 = da_input['lon'].data.flatten()
tmax_nlat = len(lat)
tmax_nlon = len(lon)

# XY of overlay plot
resources = Ngl.Resources()
resources.sfXCStartV = min(lon)
resources.sfXCEndV   = max(lon)
resources.sfYCStartV = min(lat)
resources.sfYCEndV   = max(lat)

# fill missing values as needed
if hasattr(temp,"_FillValue"):
    resources.sfMissingValueV = temp[0]._FillValue

# color map (sensitive to smaller values)
cmap = ['Ghostwhite', 'Grey0', 
        'Purple4', 'RoyalBlue3', 'RoyalBlue1', 'DeepSkyBlue', 'Cyan', 'DarkTurquoise', 
        'SeaGreen2', 'SpringGreen', 'Chartreuse', 'GreenYellow', 'OliveDrab2', 'Yellow', 
        'Gold', 'Orange', 'DarkOrange', 'Red', 'Red']

# plot styling
resources.wkColorMap = cmap
Ngl.set_values(wks2,resources)
resources.cnLevelSelectionMode = "ManualLevels"
resources.cnMinLevelValF       = 0
resources.cnMaxLevelValF       = 10
resources.cnLevelSpacingF      = 1
resources.tiXAxisString = "~F25~Longitude"
resources.tiYAxisString = "~F25~Latitude"
resources.cnFillOn              = True    
resources.cnLineLabelsOn        = False  
resources.cnInfoLabelOn         = False  
resources.nglSpreadColorEnd     = -2
resources.mpProjection = "LambertEqualArea" 
resources.mpCenterLonF = (lon[tmax_nlon-1] + lon[0])/2
resources.mpCenterLatF = (lat[tmax_nlat-1] + lat[0])/2
resources.mpLimitMode = "LatLon"
resources.mpMinLonF   = min(lon2)
resources.mpMaxLonF   = max(lon2)
resources.mpMinLatF   = min(lat2)
resources.mpMaxLatF   = max(lat2)
resources.mpOutlineBoundarySets = "GeophysicalAndUSStates"
resources.mpPerimOn   = True
resources.cnLinesOn   = False
resources.pmTickMarkDisplayMode = "Never"
resources.tiMainString = "~F26~Wrfcf, Livneh RMSE, 2017-02-05 to 2017-02-17, NG-GR-NQ-BD"
resources.lbOrientation  = "Horizontal"
resources.lbTitleString  = "~F25~Temp (C)" 

# plot
map = Ngl.contour_map(wks2,temp,resources)

# clean up
del map
del resources
Ngl.end()

# display plot
display.Image("./17_rmse_NG-GR-NQ-BD.png")

fin.